In [1]:
#import python libraries
import pandas as pd
import numpy as np

import sys
import math
import csv
import urllib
import collections

#import Scikit-Learn libraries for implementation of machine learning algorithms 
import sklearn

In [2]:
# Recursive Feature Elimination
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
#feature importance
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
#to create predictions
from sklearn.cross_validation import train_test_split
#algorithms tested
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn import tree
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Data Visualization

In [3]:
#Import the csv data
EPL_data = pd.read_csv('Training_Data1.csv')
EPL_data.head()

,ID,Year,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,HF,AF,HC,AC,HY,AY,HR,AR,Winner,Loser
0,1,2005,13-Aug-05,Everton,Man United,0,2,A,0,1,...,15,14,8,6,3,1,0,0,Man United,Everton
1,2,2005,13-Aug-05,Man City,West Brom,0,0,D,0,0,...,13,11,3,6,2,3,0,0,None,None
2,3,2005,14-Aug-05,Arsenal,Newcastle,2,0,H,0,0,...,15,17,8,3,0,1,0,1,Arsenal,Newcastle
3,4,2005,20-Aug-05,Newcastle,West Ham,0,0,D,0,0,...,9,11,10,2,1,1,0,1,None,None
4,5,2005,21-Aug-05,Chelsea,Arsenal,1,0,H,0,0,...,17,21,3,7,2,3,0,0,Chelsea,Arsenal


In [4]:
#get list of teams in EPL that play in January 2018
#team_names.head()
team_names = pd.read_csv('Team_Names.csv')
teamList = team_names['Team_Name'].tolist()
team_names.tail()

,Team_Name
11,Watford
12,Newcastle
13,Tottenham
14,Liverpool
15,Bournemouth


In [5]:
#test
print (team_names[team_names['Team_Name'] == 'Arsenal'])

  Team_Name
0   Arsenal


In [6]:
#test
print (team_names[team_names['Team_Name'] == 'Liverpool'])

    Team_Name
14  Liverpool


### Feature Creation
Use a Support Vector Machine (SVM). Plot each data item as a point in a n-dimensional environment, where each feature is a value of a particular coordinate. Find the distance (subtraction) between two vectors. 

In [7]:
#get annual vectors for each team
def getAnnualTeamData(teamName, year):
    
    annual_data = EPL_data[EPL_data['Year'] == year]
    
    # num goals scored in wins and losses
    gamesHome = annual_data[annual_data['HomeTeam'] == teamName] 
    totalGoalsScored = gamesHome['FTHG'].sum()
    gamesAway = annual_data[annual_data['AwayTeam'] == teamName]
    totalGames = gamesHome.append(gamesAway)
    numGames = len(totalGames.index)
    #total goals scored
    totalGoalsScored += gamesAway['FTAG'].sum()
    # total goals allowed
    totalGoalsAllowed = gamesHome['FTAG'].sum()
    totalGoalsAllowed += gamesAway['FTHG'].sum()
    
    #discipline: total red cards, total yellow cards
    totalYellowCards = gamesHome['HY'].sum()
    totalYellowCards += gamesAway['AY'].sum()
    totalRedCards = gamesHome['HR'].sum()
    totalRedCards += gamesAway['AR'].sum()
    
    #total fouls
    totalFouls = gamesHome['HF'].sum()
    totalFouls += gamesAway['AF'].sum()
    
    #total Corners
    totalCorners = gamesHome['HC'].sum()
    totalCorners += gamesAway['AC'].sum()

    #shots per game (spg) = total shots / total games 
    totalShots = gamesHome['HS'].sum()
    # avg shots per game
    totalShots += gamesAway['AS'].sum()
    if numGames != 0:
        spg = totalShots / numGames
    # avg shots allowed per game
    totalShotsAgainst = gamesHome['AS'].sum()
    totalShotsAgainst += gamesAway['HS'].sum()
    if numGames != 0:
        sag = totalShotsAgainst / numGames
    
    #Games Won Percentage = Games Won / (Games Won + Games Lost) 
    gamesWon = annual_data[annual_data['Winner'] == teamName] 
    gamesLost = annual_data[annual_data['Loser'] == teamName] 
    numGamesWon = len(gamesWon.index)
    numGamesLost = len(gamesLost.index)
    if numGames != 0:
        gamesWonPercentage = numGamesWon / numGames
    
    #Defense stats
        #Goalie Saves = Shots on Goal - Goal Scored
    totalShotsOnGoal = gamesHome['HST'].sum()
    totalShotsOnGoal += gamesAway['AST'].sum()
    goalieSaves = totalShotsOnGoal - totalGoalsAllowed
    
        #Saves Percentage = Goalie Saves / Shots on Goal   
    if totalShotsOnGoal != 0:
        savesPercentage = goalieSaves / totalShotsOnGoal
        
        #Saves Ratio = Shots On Goal / Goalie Saves    
    if goalieSaves != 0:
        savesRatio = totalShotsOnGoal / goalieSaves

    #Offense stats
        #Scoring Percentage = (Scoring Attempts - Goals Scored ) / Scoring Attempts
    if totalShots != 0:
        scoringPercentage = (totalShots - totalGoalsScored) / totalShots
        
        #Scoring Ratio = Shots On Goal / Goals Scored
    if totalGoalsScored != 0:
        scoringRatio = totalShotsOnGoal / totalGoalsScored       
        
            
    if numGames == 0: #if team not in dataset
        gamesWon = 0
        gamesLost = 0
        totalGoalsScored = 0
        totalGoalsAllowed = 0
        totalYellowCards = 0
        totalRedCards = 0
        totalFouls = 0
        totalCorners = 0
        spg = 0
        sag = 0
        gamesWonPercentage = 0
        goalieSaves = 0
        savesPercentage = 0
        savesRatio = 0
        scoringPercentage = 0
        scoringRatio = 0 
        
    return [totalGoalsScored, totalGoalsAllowed, totalYellowCards, totalRedCards,
        totalFouls,totalCorners, spg, sag, gamesWonPercentage, goalieSaves, savesPercentage, savesRatio,
        scoringPercentage, scoringRatio]

In [8]:
#test
getAnnualTeamData('Arsenal', 2017)

[32,
 24,
 29,
 2,
 171,
 110,
 13.647058823529411,
 12.823529411764707,
 0.5294117647058824,
 62,
 0.7209302325581395,
 1.3870967741935485,
 0.8620689655172413,
 2.6875]

In [9]:
#test
getAnnualTeamData('Chelsea', 2015)

[40,
 42,
 61,
 5,
 324,
 173,
 13.612903225806452,
 11.870967741935484,
 0.3225806451612903,
 100,
 0.704225352112676,
 1.42,
 0.9052132701421801,
 3.55]

In [10]:
#create a dictionary for all the team stats in a year for all the teams
def createAnnualDict(year):
    annualDictionary = collections.defaultdict(list)
    for team in teamList:
        team_vector = getAnnualTeamData(team, year)
        annualDictionary[team] = team_vector
    return annualDictionary

In [11]:
createAnnualDict(2016)

defaultdict(list,
            {'Arsenal': [58,
              35,
              40,
              3,
              310,
              165,
              14.193548387096774,
              12.0,
              0.4838709677419355,
              117,
              0.7697368421052632,
              1.2991452991452992,
              0.8681818181818182,
              2.6206896551724137],
             'Burnley': [15,
              27,
              25,
              0,
              171,
              58,
              8.9375,
              20.625,
              0.3125,
              19,
              0.41304347826086957,
              2.4210526315789473,
              0.8951048951048951,
              3.066666666666667],
             'Everton': [35,
              43,
              49,
              4,
              334,
              171,
              12.125,
              13.84375,
              0.25,
              90,
              0.6766917293233082,
              1.4777777777777779,
      

# Model Training
Create training method that takes in a dictionary with with all the teams vectors by year. For each game, the function calculates the difference between between the team vectors for that year. Then, the function assigns a yTrain that is a 1 if the home team wins, and 0 otherwise. The difference vector becomes the input (xTrain) for the model, and a label (yTrain).

In [12]:
def getTrainingData(years):
    totalNumGames = 0
    for year in years:
        annual = EPL_data[EPL_data['Year'] == year]
        totalNumGames += len(annual.index)
    numFeatures = len(getAnnualTeamData('Arsenal',2015)) #random team, to find dimensionality
    xTrain = np.zeros(( totalNumGames, numFeatures))
    yTrain = np.zeros(( totalNumGames ))
    indexCounter = 0
    for year in years:
        team_vectors = createAnnualDict(year)
        annual = EPL_data[EPL_data['Year'] == year]
        numGamesInYear = len(annual.index)
        xTrainAnnual = np.zeros(( numGamesInYear, numFeatures))
        yTrainAnnual = np.zeros(( numGamesInYear ))
        counter = 0
        for index, row in annual.iterrows():
            h_team = row['HomeTeam']
            h_vector = team_vectors[h_team]
            a_team = row['AwayTeam']
            a_vector = team_vectors[a_team]
            diff = [a - b for a, b in zip(h_vector, a_vector)]
            if (counter % 2 == 0):
                if len(diff) != 0:
                    xTrainAnnual[counter] = diff
                yTrainAnnual[counter] = 1
            # the opposite of the difference of the vectors should be a true negative, where team 1 does not win
            else:
                if len(diff) != 0:
                    xTrainAnnual[counter] = [ -p for p in diff]
                yTrainAnnual[counter] = 0
            counter += 1
        xTrain[indexCounter:numGamesInYear+indexCounter] = xTrainAnnual
        yTrain[indexCounter:numGamesInYear+indexCounter] = yTrainAnnual
        indexCounter += numGamesInYear
    return xTrain, yTrain

In [13]:
#get the dictionary
years = range(2005,2017)
xTrain, yTrain = getTrainingData(years)
np.save('xTrain', xTrain)
np.save('yTrain', yTrain)

In [14]:
xTrain.shape

(1656, 14)

In [15]:
yTrain.shape

(1656,)

## Feature Selection

Feature selection of the 14 features is done through recursive feature elimination and a ranking of feature importance with extra trees classifier. 10 features of the 14 were determined to be much more influential. 

In [16]:
model1 = LogisticRegression() #recursive feature elimination 
model2 = ExtraTreesClassifier() #feature importance

In [17]:
# create the RFE model and select 3 attributes
rfe = RFE(model1, 9)
model1 = LogisticRegression() #recursive feature elimination 
model2 = ExtraTreesClassifier() #feature importance
rfe = rfe.fit(xTrain, yTrain)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[ True  True False False False False  True  True  True False  True  True
  True  True]
[1 1 3 6 4 5 1 1 1 2 1 1 1 1]


In [18]:
# Feature Importance
#Top Features: total goals scored, total goals allowed, total yellow cards/red cards, total fouls, total corners,
    #spg, sag, goalie saves, scoring ratio
    
#Lowest features: scoring percentage, saves percentage, games won percentage, save ratio
    
#fit an Extra Trees model to the data
model2.fit(xTrain, yTrain)
#display the relative importance of each attribute
print(model2.feature_importances_) #the higher the more important the feature

[0.07188757 0.0692094  0.07583035 0.07054049 0.07136801 0.069479
 0.08170737 0.07113164 0.07286239 0.07049223 0.06591555 0.07451312
 0.06560475 0.06945813]


### Updated Functions to include only top 10 features

In [19]:
#updated function to include only top 10 features
def getAnnualTeamData2(teamName, year):
    
    annual_data = EPL_data[EPL_data['Year'] == year]
    
    # num goals scored in wins and losses
    gamesHome = annual_data[annual_data['HomeTeam'] == teamName] 
    totalGoalsScored = gamesHome['FTHG'].sum()
    gamesAway = annual_data[annual_data['AwayTeam'] == teamName]
    totalGames = gamesHome.append(gamesAway)
    numGames = len(totalGames.index)
    #total goals scored
    totalGoalsScored += gamesAway['FTAG'].sum()
    # total goals allowed
    totalGoalsAllowed = gamesHome['FTAG'].sum()
    totalGoalsAllowed += gamesAway['FTHG'].sum()
    
    #discipline: total red cards, total yellow cards
    totalYellowCards = gamesHome['HY'].sum()
    totalYellowCards += gamesAway['AY'].sum()
    totalRedCards = gamesHome['HR'].sum()
    totalRedCards += gamesAway['AR'].sum()
    
    #total fouls
    totalFouls = gamesHome['HF'].sum()
    totalFouls += gamesAway['AF'].sum()
    
    #total Corners
    totalCorners = gamesHome['HC'].sum()
    totalCorners += gamesAway['AC'].sum()

    #shots per game (spg) = total shots / total games 
    totalShots = gamesHome['HS'].sum()
    # avg shots per game
    totalShots += gamesAway['AS'].sum()
    if numGames != 0:
        spg = totalShots / numGames
    # avg shots allowed per game
    totalShotsAgainst = gamesHome['AS'].sum()
    totalShotsAgainst += gamesAway['HS'].sum()
    if numGames != 0:
        sag = totalShotsAgainst / numGames
    
    #Games Won Percentage = Games Won / (Games Won + Games Lost) 
    gamesWon = annual_data[annual_data['Winner'] == teamName] 
    gamesLost = annual_data[annual_data['Loser'] == teamName] 
    numGamesWon = len(gamesWon.index)
    numGamesLost = len(gamesLost.index)
    if numGames != 0:
        gamesWonPercentage = numGamesWon / numGames
    
    #Defense stats
        #Goalie Saves = Shots on Goal - Goal Scored
    totalShotsOnGoal = gamesHome['HST'].sum()
    totalShotsOnGoal += gamesAway['AST'].sum()
    goalieSaves = totalShotsOnGoal - totalGoalsAllowed
    
        #Saves Percentage = Goalie Saves / Shots on Goal   
    if totalShotsOnGoal != 0:
        savesPercentage = goalieSaves / totalShotsOnGoal
        
        #Saves Ratio = Shots On Goal / Goalie Saves    
    if goalieSaves != 0:
        savesRatio = totalShotsOnGoal / goalieSaves

    #Offense stats
        #Scoring Percentage = (Scoring Attempts - Goals Scored ) / Scoring Attempts
    if totalShots != 0:
        scoringPercentage = (totalShots - totalGoalsScored) / totalShots
        
        #Scoring Ratio = Shots On Goal / Goals Scored
    if totalGoalsScored != 0:
        scoringRatio = totalShotsOnGoal / totalGoalsScored       
        
            
    if numGames == 0: #team not in dataset
        totalGoalsScored = 0
        totalGoalsAllowed = 0
        totalYellowCards = 0
        totalRedCards = 0
        totalFouls = 0
        totalCorners = 0
        spg = 0
        sag = 0
        goalieSaves = 0
        scoringRatio = 0
        
    return [totalGoalsScored, totalGoalsAllowed, totalYellowCards, totalRedCards,
        totalFouls,totalCorners, spg, sag, goalieSaves, scoringRatio]

In [20]:
#test
getAnnualTeamData2('Chelsea', 2015)

[40, 42, 61, 5, 324, 173, 13.612903225806452, 11.870967741935484, 100, 3.55]

In [21]:
#updated functions to include only top 10 features
def createAnnualDict2(year):
    annualDictionary = collections.defaultdict(list)
    for team in teamList:
        team_vector = getAnnualTeamData2(team, year)
        annualDictionary[team] = team_vector
    return annualDictionary

In [22]:
createAnnualDict2(2016)

defaultdict(list,
            {'Arsenal': [58,
              35,
              40,
              3,
              310,
              165,
              14.193548387096774,
              12.0,
              117,
              2.6206896551724137],
             'Burnley': [15,
              27,
              25,
              0,
              171,
              58,
              8.9375,
              20.625,
              19,
              3.066666666666667],
             'Everton': [35, 43, 49, 4, 334, 171, 12.125, 13.84375, 90, 3.8],
             'Leicester': [47,
              35,
              48,
              4,
              359,
              186,
              12.25,
              14.28125,
              90,
              2.6595744680851063],
             'Man City': [62,
              40,
              54,
              3,
              348,
              210,
              15.46875,
              8.9375,
              122,
              2.6129032258064515],
             'Southa

In [23]:
#updated functions to include only top 10 features
def getTrainingData2(years):
    totalNumGames = 0
    for year in years:
        annual = EPL_data[EPL_data['Year'] == year]
        totalNumGames += len(annual.index)
    numFeatures = len(getAnnualTeamData2('Arsenal',2015)) #random team, to find dimensionality
    xTrain2 = np.zeros(( totalNumGames, numFeatures))
    yTrain2 = np.zeros(( totalNumGames ))
    indexCounter = 0
    for year in years:
        team_vectors = createAnnualDict2(year)
        annual = EPL_data[EPL_data['Year'] == year]
        numGamesInYear = len(annual.index)
        xTrainAnnual = np.zeros(( numGamesInYear, numFeatures))
        yTrainAnnual = np.zeros(( numGamesInYear ))
        counter = 0
        for index, row in annual.iterrows():
            h_team = row['HomeTeam']
            h_vector = team_vectors[h_team]
            a_team = row['AwayTeam']
            a_vector = team_vectors[a_team]
            diff = [a - b for a, b in zip(h_vector, a_vector)]
            if (counter % 2 == 0):
                if len(diff) != 0:
                    xTrainAnnual[counter] = diff
                if h_team == row['Winner']:
                    yTrainAnnual[counter] = 1
                else: 
                    yTrainAnnual[counter] = 0
            # the opposite of the difference of the vectors should be a true negative, where team 1 does not win
            else:
                if len(diff) != 0:
                    xTrainAnnual[counter] = [ -p for p in diff]
                yTrainAnnual[counter] = 0
            counter += 1
        xTrain2[indexCounter:numGamesInYear+indexCounter] = xTrainAnnual
        yTrain2[indexCounter:numGamesInYear+indexCounter] = yTrainAnnual
        indexCounter += numGamesInYear
    return xTrain2, yTrain2

In [24]:
#get the dictionary
years = range(2005,2017)
xTrain2, yTrain2 = getTrainingData2(years)
np.save('xTrain2', xTrain2)
np.save('yTrain2', yTrain2)

In [25]:
xTrain2.shape

(1656, 10)

In [26]:
yTrain2.shape

(1656,)

In [27]:
print (xTrain2)

[[ -8.          15.           3.         ...   4.05555556 -25.
    5.13846154]
 [  5.           5.         -15.         ...   4.98888889 -14.
   -5.5       ]
 [ -2.          -7.           0.         ...  -4.36363636   1.
    0.85714286]
 ...
 [ -2.          -2.           9.         ...  -1.3125     -20.
   -0.26209677]
 [  5.         -11.         -18.         ...  -2.96875     50.
    0.6037296 ]
 [ -4.          -6.          -8.         ...  -2.         -19.
   -0.33888889]]


## Model Validation
Training all 14 features using linear Regression produced the most reliable results. Thus although the function gave the top 10 features to be most influential, all 14 were used to calculate the final results for optimal accuracy

In [28]:
# Tried all the following models. Uncomment model to try.

lm = linear_model.LinearRegression()
#lm = tree.DecisionTreeClassifier()
#lm = tree.DecisionTreeRegressor()
#lm = linear_model.LogisticRegression()
#lm = linear_model.BayesianRidge()
#lm = linear_model.Lasso()
#lm = svm.SVC()
#lm = svm.SVR()
#lm = linear_model.Ridge(alpha = 0.5)
#lm = AdaBoostClassifier(n_estimators=100)
#lm = GradientBoostingClassifier(n_estimators=100) 
#lm = GradientBoostingRegressor(n_estimators=100, max_depth=9) 
#lm = RandomForestClassifier(n_estimators=100) 
#lm = KNeighborsClassifier(n_neighbors=60) #not possible with only 10 features

In [29]:
#use this
xTrain, X_test, yTrain, y_test = train_test_split(xTrain, yTrain)
print (xTrain.shape, yTrain.shape)
print (X_test.shape, y_test.shape)
#lm = linear_model.LinearRegression()
model2 = lm.fit(xTrain, yTrain)
predictions = lm.predict(X_test)
#avg pred
print (sum(predictions)/len(predictions))
#print predictions

(1242, 14) (1242,)
(414, 14) (414,)
0.5075995376244039


## Results
I tested out the above models and selected the one with the highest prediction accuracy (linear regression). I then used this model to calculate the predictions for the 2018 games. 

In [30]:
def createGamePrediction(team1_vector, team2_vector, xTrain, yTrain):
    xTrain, X_test, yTrain, Y_test = train_test_split(xTrain, yTrain)
    xTrain.shape, yTrain.shape
    X_test.shape, y_test.shape
    lm = linear_model.LinearRegression()
    model2 = lm.fit(xTrain, yTrain)
    diff = [a - b for a, b in zip(team1_vector, team2_vector)]
    predictions = lm.predict([diff])
    print(diff)
    return predictions

In [31]:
team1_vector = getAnnualTeamData("Liverpool", 2017)
team2_vector = getAnnualTeamData("Liverpool", 2017)
print ('Probability that home team wins:',createGamePrediction(team1_vector, team2_vector,xTrain, yTrain))

[0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.0]
Probability that home team wins: [0.51099958]
